# Project: Fleet Quality Analysis For Food Delivery App.

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#Quest">Research Points</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#clean">Data Cleaning</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
<li><a href="#Actions">Suggested Actions</a></li>
    
</ul>

<a id='intro'></a>
## Introduction




> **Background:**
This investigation is for a food delivery application that helps people discover and order the food they will love. We have been largely focused on the food delivery and online ordering experience and our traction has been exponential. We aim to grow through a diverse set of channels while maintaining healthy economics.


> **The dataset description:**

* Driver_Id: ID.
* Date: Order date.
* Pool: Delivery area.
* Delivery_Time: The duration from customer request till order arrived.
* Trip_Duration: The duration from runner acceptance till order arrived.
* Pending_to_Dispatch: The duration from dispatch the order by user till accepted by runner.
* FM: The duration from runner acceptance till arrived at restaurant.
* EFM: The estimated duration from runner acceptance till arrived at restaurant through google map.
* Waiting_Time: Waiting time in restaurant after driver arrived and before receiving order.
* LM: The duration from receiving the order from the restaurant till order arrived.
* ELM : The estimated duration from receiving the order from the restaurant till order arrived.
* Delivered: Delivery status (0 or 1).
* Total_Distance: Total distance between restaurant and customer address.
* Hour: Order hour.
* Late_FM: EFM - FM.
* Late_LM: ELM - LM.
* Total_Late: Late_FM + Late_LM.
* Lateness_status: late or not (1 or 0).

<a id='Quest'></a>
## **Research Points:**


1. Delivery duration Per pool.
2. The percentage of late trip per pool.
3. The percentage of violators with late trips per pool.
4. Your Action plan to enhance the violators % per pool.
5. Initiatives to decrease overall delivery duration.
6. Initiatives to automate our runners onboarding process.   

### importing libraries that will be used to investigate Dataset

In [76]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import datetime
sns.set_style("whitegrid")

<a id='wrangling'></a>
## Data Wrangling

 **This is a three step process:**

*  Gathering the data from Dataset and investegate it trying to understand more details about it. 


*  Assessing data to identify any issues with data types, structure, or quality.


*  Cleaning data by changing data types, replacing values, removing unnecessary data and modifying Dataset for easy and fast analysis.


### Gathering Data

In [77]:
# loading CSV files in to 3 Dataframes  //patients, treatments and adverse reaction//

data = pd.read_csv("../input/row-data/Row Data.csv")

In [78]:
#checking 5 rows sample from Dataframes

data.head(5)

In [79]:
data.columns

In [80]:
# Select important features 

X =  ['driver_uuid','pending_time','Pool', 
      'Delivery Time','Trip_Duration','Pending_to_Dispatch',
      'First Mile', 'EFM', 'Waiting Time', 'Last Mile','ELM', 
      'OFD_Arrived', 'Arrived_to_Delivered','SR%', 'Total_Distance',"hour" ]

In [81]:
# make dataframe from important variables only

df = data[X].copy()

In [82]:
df.head(5)

### Assessing Data

In [83]:
#checking Dataframe basic informations (columns names, number of values, data types ......)

df.info(memory_usage="deep")

In [84]:
#checking Dataframe shape (number of rows and columns)

df.shape

In [85]:
#checking more information and descriptive statistics

df.describe()

**From above summary we can see that:**
> * the maximum value is so large in almost all columns.
> * the minmum values is upnormal in many columns.

**So we need to check outliers, typing mistakes and any thing affects these values.**

In [86]:
# checking for NaN values patients

df.isnull().sum()

**There are alot of empty cells in this dataset which may affect our statistics accurecy. We can not ignore,drop or replace all cells with empty values.**

In [87]:
#checking for duplicated rows 

df[df.duplicated(keep=False)]

In [88]:
# Check number of unique values

df.nunique()

<a id='clean'></a>

## Data Cleaning 


### <font color='blue'>Tidiness issues</font>


1. Extract date from pending_time and change pending_time data type into date 


### <font color='blue'>Quality issues</font>

1. Change columns names (remove spaces and make names more descriptive) for readability and accessibility.
2. remove duplicates 
3. Add 3 columns LM_Late, FM_Late and Total_Late.
4. change  SR% column values into 0 and 1 then make column data type boolean.
5. Check outliers and drop the upnormal values
6. Add new column for lateness status
7. Add new columns for Day and Month.

In [89]:
# make copy of origenal dataframes to clean them

df_clean = df.copy()

### <font color='blue'>Tidiness issues</font>

#### 1. Extract date from pending_time and change pending_time data type into date

In [90]:
df_clean.pending_time = df_clean.pending_time.apply(lambda x:x.split(" ")[0])

In [91]:
df_clean.pending_time = pd.to_datetime(df_clean.pending_time )

In [92]:
# confirm changes

df_clean.pending_time.sample(5)

In [93]:
df_clean.pending_time.dtype

### <font color='blue'>Quality issues</font>

#### 1. Change columns names (remove spaces and make names more descriptive) for readability and accessibility.

In [94]:
df_clean.columns

In [95]:
col_dic = {'driver_uuid':'Driver_Id', 
           'pending_time':"Date", 
           'Pool':"Pool",
           'Delivery Time':"Delivery_Time",
           'Trip_Duration': "Trip_Duration",
           'Pending_to_Dispatch':"Pending_to_Dispatch",
           'First Mile':"FM", 
           'EFM':"EFM", 
           'Waiting Time':"Waiting_Time",
           'Last Mile':"LM",
           'ELM':"ELM",
           'OFD_Arrived':"OFD_Arrived",
           'Arrived_to_Delivered':"Arrived_to_Delivered",
           'SR%':"Delivered", 
           'Total_Distance':"Total_Distance","hour":"Hour"}

In [96]:
df_clean.rename(columns=col_dic, inplace=True)

In [97]:
# confirm changes

df_clean.columns

#### 2. remove duplicates

In [98]:
df_clean.drop_duplicates(inplace=True)

In [99]:
# confirm changes

df_clean.duplicated().sum()

#### 3. Add 3 columns LM_Late, FM_Late and Total_Late

In [100]:
df_clean["Late_FM"] = df_clean["FM"] - df_clean["EFM"]
df_clean["Late_LM"] = df_clean["LM"] - df_clean["ELM"]
df_clean["Total_Late"] = df_clean["Late_FM"] + df_clean["Late_LM"]

In [101]:
# confirm changes

df_clean[["Late_FM", "Late_LM", "Total_Late"]].head()

#### 4. change SR% column values into 0 and 1 then make column data type boolean.

In [102]:
df_clean.Delivered.value_counts()

In [103]:
df_clean.Delivered = df_clean.Delivered.replace({100:1})

In [104]:
df_clean.Delivered = df_clean.Delivered.astype("bool")

In [105]:
# confirm changes

df_clean.Delivered.value_counts()

In [106]:
df_clean.Delivered.dtype

In [107]:
df_clean.Delivered.mean()

#### 5.Check outliers and drop the upnormal values

In [108]:
def outliers(df, col, asc):
    """ This function is to select outliers from dataframe on 
    a selected column then sort the result Ascending or descending 

    Args:
        df (dataframe): pandas dataframe
        col (string): column name
        asc (boolean): True or False

    Returns:
        dataframe: sorted data frame Ascending or descending that includes outliers of selected column.
    """
    mean = df[col].mean()
    std =  df[col].std()
    cut_off = std*3
    lower, upper = mean -cut_off , mean +cut_off
    outliers = df[(df[col] <lower)|(df[col] >upper)]
    outliers = outliers.sort_values(col, ascending=asc)

    return outliers

In [109]:
# Check outliers on Delivery_Time


outliers(df_clean,'Delivery_Time', False ).head(15)

**As we can see there are many observations with delivery time more than 200 minutes which can not be correct so We will drop those observations**

In [113]:
long = df_clean[df_clean.Delivery_Time >200]
df_clean.drop(long.index, inplace=True)

In [114]:
# confirm changes 

long

#### 6. Add new column for lateness status

##### Code

In [115]:
df_clean["Lateness_status"] = np.where(np.logical_or(df_clean.Late_FM >=5,df_clean.Late_LM>=5),1, 0)

In [116]:
# Check 
df_clean.sample(5)

#### 7.Add new columns for Day and Month.

In [117]:
df_clean["Month"] = df_clean.Date.dt.strftime("%Y-%m")
df_clean["Day"] = df_clean.Date.dt.strftime("%w")

In [118]:
# confirm changes 

df_clean[["Month", "Day"]].sample(5)


<a id='eda'></a>
## Exploratory Data Analysis

> Now I'm going to explore this data and try to find patterns in it, compute statistics and visualize the relationships
to answer the questions and detect the characteristics that affect patients attending to their hospital appointment


In [119]:
 df_clean.columns.values

In [120]:
quantitative_features = [i for i in df_clean.columns.values if str(df_clean[i].dtype)[:3] in ("flo", "int")]
len(quantitative_features)

In [121]:
# Quantitative Variables Density plots

sns.set_style("whitegrid")

fig,ax = plt.subplots(4,4,figsize=(25,12))
k=0
j=0
for col in quantitative_features:
    sns.kdeplot(df_clean[col], ax=ax[k,j],
                shade=True,
                color='#2f5586', edgecolor='black',
                linewidth=1.5, alpha=0.9,
                zorder=3,
               )
    
    ax[k,j].set_xlabel(col, fontsize=17, color="k")
    ax[k,j].set_ylabel("Density", fontsize=17, color="k")
    if j>=3:
        k+=1
        j=-1
    j+=1
plt.subplots_adjust(hspace = 0.4, wspace=0.2)
fig.suptitle('Quantitative Variables Density plots', fontsize=25, color="k");

In [122]:
# Quantitative Variables Density plots

sns.set_style("whitegrid")

fig,ax = plt.subplots(4,4,figsize=(25,12))
k=0
j=0
for col in quantitative_features:
    sns.histplot(df_clean[col], ax=ax[k,j],
                #shade=True,
                color='#2f5586', edgecolor='black',
                #linewidth=1.5, alpha=0.9,
                #zorder=3,
               )
    
    ax[k,j].set_xlabel(col, fontsize=17, color="k")
    ax[k,j].set_ylabel("Frequency", fontsize=17, color="k")
    if j>=3:
        k+=1
        j=-1
    j+=1
plt.subplots_adjust(hspace = 0.4, wspace=0.2)
fig.suptitle('Quantitative Variables Histograms', fontsize=25, color="k");

### From above visualizations we can see that:
    
    - Delivery time distribution is right skewed and that means the orders with high delivery time have the less frequency.
    - Trip Duration, LM and FM have right skewed distribution also.

In [157]:
# Quantitative Variables Boxplots

fig = plt.figure(figsize=(25,6))
sns.boxplot(data=df_clean[quantitative_features],saturation=.5, palette="viridis", showfliers = False)
plt.title("Quantitative Variables Summary Statistics", fontsize=25, color="k", pad=20)
plt.xticks(fontsize= 13, rotation=35);

### From above visualizations we can see that:
    
    - 75% from orders have Delivery time less than 55 Minutes.
    - 75% from orders have Trip Duration less than 45 Minutes.
    - 25% from orders have total late more than 12 Minutes.

### Research Point 1
> ####  Delivery duration Per pool.
    
            - Trip_Duration Average.
            - Delivery_Time Average.

In [124]:
# Average Delivery_Time and Trip_Duration per pool

avg_duration = df_clean.groupby("Pool")[["Trip_Duration", "Delivery_Time"]].mean().sort_values("Trip_Duration",ascending=False)
avg_duration

In [125]:
best_five_pool =avg_duration.sort_values("Trip_Duration", ascending=False).iloc[:5]
best_five_pool

In [126]:
c1 = sns.color_palette("Oranges",8)[3:]
c2 = sns.color_palette("Blues", 8)[3:]

delivery_avg = best_five_pool["Delivery_Time"].sort_values(ascending=True)
delivery_avg.plot(kind="bar",
                      rot = 0,
                      width = 0.40,
                      color=c1,
                      alpha = 0.8,
                      fontsize = 12,
                      figsize=(14,8)
                    )
    
trip_avg = best_five_pool["Trip_Duration"].sort_values(ascending=True)
trip_avg.plot(kind="bar",
              rot = 0,
              width = 0.40,
              color=c2,
              alpha = 0.8,
              fontsize = 12,
              figsize=(14,8)
                )         

plt.ylabel("Average Durations in Minutes",fontsize=16, color="k", labelpad=15)
plt.xlabel("Pool",fontsize=16, color="k", labelpad=15)
plt.title("Average Durations per pool (Best 5)", fontsize=18, color="k", pad=20);


plt.legend(fontsize=14);

In [127]:
worst_five_pool =avg_duration.sort_values("Trip_Duration", ascending=True).iloc[:5]
worst_five_pool

In [128]:
c1 = sns.cubehelix_palette(8)[3:]
c2 = sns.color_palette("crest", 10)[5:]

delivery_avg = worst_five_pool["Delivery_Time"].sort_values(ascending=True)
delivery_avg.plot(kind="bar",
                      rot = 0,
                      width = 0.40,
                      color=c1,
                      alpha = 0.8,
                      fontsize = 12,
                      figsize=(14,8)
                    )
    
trip_avg = worst_five_pool["Trip_Duration"].sort_values(ascending=True)
trip_avg.plot(kind="bar",
              rot = 0,
              width = 0.40,
              color=c2,
              alpha = 0.8,
              fontsize = 12,
              figsize=(14,8)
                )         

plt.ylabel("Average Durations in Minutes",fontsize=16, color="k", labelpad=15)
plt.xlabel("Pool",fontsize=16, color="k", labelpad=15)
plt.title("Average Durations per pool (Worst 5)", fontsize=18, color="k", pad=20);


plt.legend(fontsize=14);

In [129]:
df_clean.groupby("Pool")[["Pending_to_Dispatch"]].mean().sort_values("Pending_to_Dispatch",ascending=False).iloc[:3]

### Result 1:

**From above visualizations we can see that:**


**1. The best 5 pools in both average Trip_Duration and average Delivery_time are:**

    - Zayed - Palm hills.
    - Madinaty.
    - Haram & Faisal.
    - Zayed - Districts.
    - Zaytoun & Ain Shams.

    

**2. The worst 5 pools in both Trip_Duration average and Delivery_time average are:**

    - Ras Sudr.
    - Tanta.
    - El Mansoura.
    - 1-Oct.
    - Hadayek El Ahram.



**3. The Pending_to_Dispatch average time is the highest in:**

    - Ras Sudr (41.81) min on average.
    - Zayed - Palm hills (16.83) min on average.
    - Shoubra (15:09) min on average.
    


### Research Point 2
> #### The percentage of late trip per pool.

In [130]:
# Lateness percentage per pool

per_delay = df_clean.groupby("Pool")["Lateness_status"].mean().sort_values()*100
per_delay

In [131]:
# Lateness Percentage per pool bar plot

c =   sns.cubehelix_palette(14)[4:]
delay = per_delay.iloc[15:].sort_values(ascending=True)
delay.plot(kind="barh",
              rot = 0,
              width = 0.7,
              color=c,
              alpha = 0.8,
              fontsize = 12,
              figsize=(14,7)
                )


plt.xlabel("Lateness Percentage %",fontsize=15, color="k", labelpad=15)
plt.ylabel("Pool",fontsize=15, color="k",labelpad=15)
plt.title("Lateness Percentage  LM or FM (Worst 10)", fontsize=18, color="k", pad=15);

### Result 2:

**. The worst ten pools on Lateness Percentage are:**

    (Late_LM or Late_FM more than or equal to 5 Minutes)

    - Alex East                      68.66 % Latness  
    - Zayed - Palm hills             65.56 % Latness 
    - Mokattam                       65.50 % Latness 
    - Hadayek El Ahram               64.25 % Latness 
    - Haram & Faisal                 64.06 % Latness 
    - Shoubra                        63.59 % Latness 
    - Maadi - Zaharaa                62.21 % Latness 
    - New Cairo - Banks              61.94 % Latness 
    - Alex West                      61.21 % Latness 
    - Zaytoun & Ain Shams            60.51 % Latness 
    

### Research Point 3

> #### The percentage of violators with late trips per pool.

In [132]:
total_drivers = df_clean.groupby("Pool")["Driver_Id"].count()
total_drivers

In [133]:
violators_per_pool = df_clean[df_clean.Lateness_status == True].groupby("Pool")["Driver_Id"].count()
violators_per_pool

In [134]:
violators_percent_pool =  (violators_per_pool / total_drivers) * 100
violators_percent_pool.sort_values(ascending=True)

In [135]:
t = pd.concat([total_drivers,violators_per_pool,violators_percent_pool], axis=1)
#t = t.reset_index
t.columns = ["Total_Drivers", "Violators_Drivers", "Violators_Percentage"]
t.sort_values("Violators_Percentage", ascending=False)

In [136]:
# Violators with Lateness Percentage per pool bar plot

c = c = sns.color_palette("Oranges", 14)[4:]

violators_per_ten = violators_percent_pool.sort_values(ascending=False).iloc[:10]
violators_per_ten.sort_values(ascending=True).plot(kind="barh",
                          rot = 0,
                          width = 0.60,
                          color=c,
                          alpha = 0.8,
                          fontsize = 12,
                          figsize=(12,8)
                                        )
                    

plt.xlabel("Violators Percentage %",fontsize=15, color="k", labelpad=15)
plt.ylabel("Pool",fontsize=15, color="k",labelpad=20)
plt.title("Violators Percentage Per Pool (Worst 10)", fontsize=18, color="k", pad=15); 

### Result 3:

**. The worst ten pools on Violators percentage in FM or LM are:**


    - Alex East             69.04 %
    - Zayed - Palm hills    66 %
    - Mokattam              65.88 %
    - Hadayek El Ahram	  64.55 %
    - Shoubra	   	    64.29 %
    - Haram & Faisal	    64.23%
    - Maadi - Zaharaa	   62.32%
    - New Cairo - Banks	 62.25%
    - Alex West	         61.36%
    - Zaytoun & Ain Shams   61.29% 




        - 	
             

### Research Point 4

> #### LM & FM Lateness Trend per Hour

In [137]:
lm_late_mean = df_clean.groupby("Hour")["Late_LM"].mean()
lm_late_mean

In [138]:
fm_late_mean = df_clean.groupby("Hour")["Late_FM"].mean()
fm_late_mean

In [139]:
lm_late_total = df_clean.groupby("Hour")["Late_LM"].sum()
lm_late_total

In [140]:
fm_late_total = df_clean.groupby("Hour")["Late_FM"].sum()
fm_late_total

In [141]:
fig,ax = plt.subplots(1,2,figsize=(25,6))



lm_late_total.plot( marker="o",
                    alpha = 1,
                    fontsize = 12,
                     ax=ax[0], label="LM_Late")

fm_late_total.plot( marker="o",
                    alpha = 1,
                    fontsize = 12,
                     ax=ax[0], label="FM_Late")

lm_late_mean.plot( marker="o",
                    alpha = 1,
                    fontsize = 12,
                     ax=ax[1], label="LM_Late")
fm_late_mean.plot( marker="o",
                    alpha = 1,
                    fontsize = 12,
                     ax=ax[1], label="FM_Late")




xlbl = np.arange(0,24)



ax[0].set_xlabel("Hour",fontsize=15, color="k", labelpad=20, ha="center")
ax[0].set_ylabel("Total Late in Minutes",fontsize=15, color="k", labelpad=20, ha="center")
ax[0].set_title("The Total Late Trend Per Hour LM & FM", fontsize=18, color="k", pad=20)
ax[0].set_xticks(np.arange(0,24,1), xlbl, fontsize=14, rotation=45)
ax[0].legend(fontsize=13, loc=9)

ax[1].set_xlabel("Hour",fontsize=15, color="k", labelpad=20, ha="center")
ax[1].set_ylabel("Average Late in Minutes",fontsize=15, color="k", labelpad=20, ha="center")
ax[1].set_title("The Average Late Trend Per Hour LM & FM", fontsize=18, color="k", pad=20)
ax[1].set_xticks(np.arange(0,24,1), xlbl, fontsize=14, rotation=45)
ax[1].legend(fontsize=13, loc=9);


In [142]:
total_late_sum = df_clean.groupby("Hour")["Total_Late"].sum()
total_late_sum

In [143]:
total_late_avg = df_clean.groupby("Hour")["Total_Late"].mean()
total_late_avg

In [144]:
fig,ax = plt.subplots(1,2,figsize=(25,6))

c = sns.color_palette("rocket")[4:5]
total_late_sum.plot( marker="o",
                    alpha = 1,
                    fontsize = 12,
                    label="Total Late", ax=ax[0])

total_late_avg.plot( marker="o",
                    alpha = 1,
                    fontsize = 12,color=c,
                     label="Average Late" , ax=ax[1])

xlbl = np.arange(0,24)



ax[0].set_xlabel("Hour",fontsize=15, color="k", labelpad=20, ha="center")
ax[0].set_ylabel("Total Late in Minutes",fontsize=15, color="k", labelpad=20, ha="center")
ax[0].set_title("The Total Late Trend Per Hour LM + FM", fontsize=18, color="k", pad=20)
ax[0].set_xticks(np.arange(0,24,1), xlbl, fontsize=14, rotation=45)

ax[1].set_xlabel("Hour",fontsize=15, color="k", labelpad=20, ha="center")
ax[1].set_ylabel("Average Late in Minutes",fontsize=15, color="k", labelpad=20, ha="center")
ax[1].set_title("The Average Late Trend Per Hour LM + FM", fontsize=18, color="k", pad=20)
ax[1].set_xticks(np.arange(0,24,1), xlbl, fontsize=14, rotation=45);


**There is no skwness in variables distributions that indecates any strong point related Lateness time.**

### Result 4:

**From above visualizations we can see that:**

    - The highest LM Lateness total at 18:00 O'clock.
    - The highest FM Lateness total at 21:00 O'clock.
    - The highest LM Lateness average at 18:00 O'clock.
    - The highest FM Lateness average at 09:00 O'clock.
    
    - The highest Lateness Total of LM + FM is at 18:00 O'clock.
    - The highest Lateness Average of LM + FM is at 21:00 O'clock.
    
**So we need to pay more effort on hours 09:00, 18:00 and 21:00 to decrease Lateness time like raise EFM and ELM time a little**

### Research Point 5

> #### Correlation Between Late_LM and Total_Distance. 
> #### Correlation Between Late_LM and Waiting_Time.
> #### Correlation Between Late_FM and Total_Distance.
> #### Correlation Between Late_FM and Waiting_Time .

In [145]:
# Check correlation with heatmap

fig, ax = plt.subplots(figsize=(25,8))
corr_matrix = df_clean.corr()
mask = np.zeros_like(corr_matrix)
mask[np.triu_indices_from(mask)] = True
sns.heatmap(corr_matrix, annot=True, mask=mask );

plt.title( 'Correlations Heatmap', fontsize=20, fontweight='bold', fontfamily='serif', pad=15);


In [146]:
# Scatter plot between Profit and (Revenue, Quantity and Cost)

c = sns.color_palette("rocket")[1:5]

fig,ax = plt.subplots(2,2,figsize=(25,10))
k=0
j=0
i = 0
for col in ["Late_FM", "Late_LM"]:
    for row in ["Total_Distance", "Waiting_Time"]:
        sns.regplot(data = df_clean, x = col, y = row, ax = ax[k,j], color= c[i])
        i+=1
        ax[k,j].set_xlabel(col, fontsize=17, color="k")
        ax[k,j].set_ylabel(row, fontsize=17, color="k")
        ax[k,j].set_title("Correlation between {} and {}".format(col, row), fontsize=18, color="k", pad=25)
        if j>=1:
            k+=1
            j=-1
        j+=1
    
#fig.suptitle('Correlation between Late_FM, Late_LM, Total_Distance and Waiting_Time', fontsize=25, color="k")
plt.subplots_adjust(hspace = 0.6, wspace=0.2);

### Result 5:

**From above visualizations we can see that:**

    - There is no strong correlation between Waiting_Time and Late_LM.
    - There is no strong correlation between Waiting_Time and Late_FM.
    - There is no strong correlation between Total_Distance and Late_LM.
    - There is no strong correlation between Total_Distance and Late_FM.

### Research Point 6

> #### Drivers Evaluation


In [147]:
total_drivers = df_clean.groupby("Pool")["Driver_Id"].count()
total_drivers

In [148]:
violators = df_clean.groupby(["Pool","Driver_Id"]).agg({"Lateness_status":"mean", "Total_Distance":"count", "Total_Late":"sum"})#.to_frame()
violators.reset_index(inplace=True)
violators_df = violators.copy()
violators_df.rename(columns={"Lateness_status":"Lateness_Percentage","Total_Distance":"Orders_Count" }, inplace=True)
violators_df["Lateness_Percentage"] = violators_df.Lateness_Percentage*100

In [149]:
violators_df

In [163]:
violators_per_pool = violators_df.groupby(["Pool"]).Lateness_Percentage.mean().sort_values(ascending=False).iloc[:10]
violators_per_pool

In [169]:
# Lateness Percentage per pool bar plot

c =   sns.color_palette("Blues", 14)[4:]
vi_ten = violators_per_pool.sort_values(ascending=True)
vi_ten.plot(kind="barh",
              rot = 0,
              width = 0.7,
              color=c,
              alpha = 0.8,
              fontsize = 12,
              figsize=(14,7)
                )


plt.xlabel("Violators Percentage %",fontsize=15, color="k", labelpad=15)
plt.ylabel("Pool",fontsize=15, color="k",labelpad=15)
plt.title("Violators Percentage per Pool (Worst 10)", fontsize=18, color="k", pad=15);

### Result 6:

**The worst ten pools on Violators percentage in LM or FM are:**

    - Alex East              72.599232 %
    - Mokattam               70.874300 %
    - Hadayek El Ahram       70.266440 %
    - Haram & Faisal         68.685392 %
    - Zayed - Palm hills     67.596099 %
    - Zaytoun & Ain Shams    67.535997 %
    - Alex West              67.484717 %
    - Heliopolis - Korba     66.215336 %
    - Maadi - Zaharaa        65.661489 %
    - New Cairo - Banks      63.384935 %

In [150]:
# violators with lateness percentage between 70 and 90 % and order count more than 10

violators_70_to_90 = violators_df[(violators_df.Lateness_Percentage >=70) &
             (violators_df.Lateness_Percentage <=90) &
            (violators_df.Orders_Count >10)].copy().to_csv("Drivers_need_coaching.csv", index=False)

In [151]:
# violators with lateness percentage more than 90 % and order count more than 10

violators_over90 = violators_df[(violators_df.Lateness_Percentage >90) &
            (violators_df.Orders_Count >10)].copy().to_csv("Drivers_need_actions.csv", index=False)

In [152]:
# violator Drivers which need coaching

Driver_to_coach = pd.read_csv("Drivers_need_coaching.csv")
Driver_to_coach.head(20).sort_values("Lateness_Percentage", ascending=False)

In [153]:
# violator Drivers which need actions


Driver_to_action = pd.read_csv("Drivers_need_actions.csv")
Driver_to_action.head(20).sort_values("Lateness_Percentage", ascending=False)

<a id='conclusions'></a>
## Conclusions


**1. Delivery time distribution is right skewed and that means the orders with high delivery time have the less frequency. Trip Duration, LM and FM also have right skewed distribution.**


**2. 75% from orders have Delivery time less than 55 Minutes.**

**3. 75% from orders have Trip Duration less than 45 Minutes.**

**4. 25% from orders have total late more than 12 Minutes.**

**5. The best 5 pools in both average Trip_Duration and average Delivery_time are:**

        - Zayed - Palm hills.
        - Madinaty.
        - Haram & Faisal.
        - Zayed - Districts.
        - Zaytoun & Ain Shams.



**6. The worst 5 pools in both Trip_Duration average and Delivery_time average are:**

        - Ras Sudr.
        - Tanta.
        - El Mansoura.
        - 1-Oct.
        - Hadayek El Ahram.



**7. The Pending_to_Dispatch average time is the highest in:**

        - Ras Sudr (41.81) min on average.
        - Zayed - Palm hills (16.83) min on average.
        - Shoubra (15:09) min on average.




**8. The worst pools on Lateness Percentage are:**

        (Late_LM or Late_FM more than or equal to 5 Minutes)

        - Alex East                      68.66 % Latness  
        - Zayed - Palm hills             65.56 % Latness 
        - Mokattam                       65.50 % Latness 


**9. The worst ten pools on Violators percentage in FM or LM are:**

        - Alex East             69.04 %
        - Zayed - Palm hills    66 %
        - Mokattam              65.88 %


**10. We need to pay more effort on hours 09:00, 18:00 and 21:00 to decrease Lateness time:**

        - The highest LM Lateness total at 18:00 O'clock.
        - The highest FM Lateness total at 21:00 O'clock.
        - The highest LM Lateness average at 18:00 O'clock.
        - The highest FM Lateness average at 09:00 O'clock.
        - The highest Lateness Total of LM + FM is at 18:00 O'clock.
        - The highest Lateness Average of LM + FM is at 21:00 O'clock.


**11. There are no strong correlations to consider between:**

        - Waiting_Time and Late_LM.
        - Waiting_Time and Late_FM.
        - Total_Distance and Late_LM.
        - Total_Distance and Late_FM.

    
**12. Alex East, Mokattam and Hadayek El Ahram Pools have the highest Violators percentage in LM or FM with 72.6 %, 70.9 % and 70.3 % respectively.**
 

<a id='Actions'></a>

## Suggested Actions


    
**1. - Every Pool manager should observe Drivers with Lateness Percentage between 70% and 90%, make fast meeting with them and try to find the reason behind high lateness percentage from total orders they delivered.**

**2. - Observe Drivers with Lateness Percentage more than 90% as well you may find another issues with their performance. Take the necessary actions with them**

**3. Collect more data about drivers to analyze their behaviour and find out the reasons behind high lateness percentage to avoid in the future**

**4.Pay more effort on hours 09:00, 18:00 and 21:00 to decrease Lateness time:**

     - Raise EFM and ELM time alittle to avoid lateness.
     - Manage breaks to not intersect with this hours.

## Limitations

**1.  There is a lot of empty values in each columns specially columns that indicates lateness which may affect our analysis accuracy.**

**2.There are typing or data entry issues. Delivery_Time couldn't be 296.63 minutes and Late_FM couldn't be - 86.8.**

**3.Finally to evaluate Drivers performance effectively we need more data about them like their behavior and working hours and customer rating.**